# CNN Example 1
For this example, we have images of cars and flowers, which have been divided into training and testing sets, and we have to build a CNN that identifies whether an image is a car or a flower.

### Step 1: Import the numpy library and the necessary Keras libraries and classes

In [1]:
import tensorflow as tf

# Print TensorFlow version for reference
print("TensorFlow version:", tf.__version__)

# List available GPU devices
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs detected:", gpus)
    # Optional: Enable memory growth to avoid allocating all GPU memory at once
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("No GPU detected, using CPU.")


2025-02-15 01:13:36.886200: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-15 01:13:36.895526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739610816.906499    6480 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739610816.909659    6480 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-15 01:13:36.921574: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

TensorFlow version: 2.18.0
GPUs detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import os
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

CUDA_VISIBLE_DEVICES: None


In [3]:
# Import the Libraries
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np
from tensorflow import random

In [4]:
import sys
print("Python executable:", sys.executable)

Python executable: /home/oem/Documents/github/magnimind_projects/magpenv/bin/python


In [5]:
# Cell 1: Set environment variable before any TensorFlow import
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Ensure GPU 0 is visible

# Cell 2: Import TensorFlow and verify GPU setup
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPUs available:", tf.config.list_physical_devices('GPU'))

# Continue with your model definition and training...

TensorFlow version: 2.18.0
Built with CUDA: True
GPUs available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
print(tf.sysconfig.get_build_info())

OrderedDict({'cpu_compiler': '/usr/lib/llvm-18/bin/clang', 'cuda_compute_capabilities': ['sm_60', 'sm_70', 'sm_80', 'sm_89', 'compute_90'], 'cuda_version': '12.5.1', 'cudnn_version': '9', 'is_cuda_build': True, 'is_rocm_build': False, 'is_tensorrt_build': False})


### Step 2: Now, set a seed and initiate the model with the `Sequential` class

In [7]:
#set a seed
seed = 1
np.random.seed(seed)
random.set_seed(seed)

# Initialising the CNN
classifier = Sequential()

### Step 3: Add the first layer of the CNN, set the input shape to (64, 64, 3), the dimension of each image, and set the activation function as a ReLU:

In [8]:
classifier.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))

/home/oem/Documents/github/magnimind_projects/magpenv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1739610818.827885    6480 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14165 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


### Step 4: Now, add the pooling layer with the image size as 2x2

In [9]:
classifier.add(MaxPool2D(pool_size=(2, 2)))

### Step 5: Flatten the output of the pooling layer by adding a flattening layer to the CNN model:

In [10]:
classifier.add(Flatten())

### Step 6: Add the first Dense layer of the MLP. 
Here, 128 is the output of the number of nodes. As a good practice, 128 is good to get started. activation is relu. As a good practice, the power of two is preferred

In [11]:
classifier.add(Dense(128, activation='relu'))

### Step 7: Add the output layer of the MLP.
This is a binary classification problem, so the size is 1 and the activation is `sigmoid`:

In [12]:
classifier.add(Dense(1, activation='sigmoid'))

### Step 8: Compile the network
Use an adam optimizer and compute the accuracy during the training process 

In [13]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Step 9: Create training and test data generators. 
- Rescale the training and test images by `1/255` so that all the values are between `0` and `1`.
- Set these parameters for the training data generators only 
 - `shear_range=0.2`, `zoom_range=0.2`, and `horizontal_flip=True`
 
 - https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html


In [14]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the directory where your images are stored
data_dir = '/home/oem/Downloads/train'  # Update this path if needed

# List all image files (assuming they end with '.jpg')
files = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]

# Create a DataFrame with filenames and labels
labels = []
for file in files:
    if file.startswith('cat'):
        labels.append('cat')
    elif file.startswith('dog'):
        labels.append('dog')
    else:
        labels.append('unknown')  # In case there are files that don't match

df = pd.DataFrame({'filename': files, 'class': labels})

# Optionally, you can filter out any unknown labels
df = df[df['class'] != 'unknown']

# Split the DataFrame into training (90%) and test (10%) sets
train_df, test_df = train_test_split(
    df, 
    test_size=0.1, 
    random_state=1,   # or any seed you prefer
    stratify=df['class']  # ensures proportional split per class
)

# Create ImageDataGenerators with your desired settings
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

### Step 10: Create a training set from the training set folder.
'training_set' is the folder where our data has been placed. Our CNN model has an image size of `64x64`, so the same size should be passed here too. `batch_size` is the number of images in a single batch, which is `32`. `Class_mode` is set to binary since we are working on binary classifiers

In [15]:
# Create generators using the DataFrame
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=data_dir,
    x_col='filename',
    y_col='class',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'  # since it's a binary classification
)

Found 22500 validated image filenames belonging to 2 classes.


### Step 11: Repeat step 10 for the test set 
while setting the folder to the location of the test images, that is, 'test_set'

In [16]:
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=data_dir,
    x_col='filename',
    y_col='class',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 2500 validated image filenames belonging to 2 classes.


### Step 12: Finally, fit the data. 
Set the `steps_per_epoch` to `STEP_SIZE_TRAIN` and the `validation_steps` to `STEP_SIZE_TEST`. 

Why do we need `steps_per_epoch` ?

Keep in mind that a Keras data generator is meant to loop infinitely — it should never return or exit.

Since the function is intended to loop infinitely, Keras has no ability to determine when one epoch starts and a new epoch begins.

Therefore, we compute the `steps_per_epoch` value as the total number of training data points divided by the batch size. Once Keras hits this step count it knows that it’s a new epoch.

In [17]:
# Calculate steps per epoch for training and testing
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_TEST  = test_generator.n // test_generator.batch_size

# Fit the model using the generators
history = classifier.fit(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    epochs=30,  # set the number of epochs as needed
    validation_data=test_generator,
    validation_steps=STEP_SIZE_TEST
)

Epoch 1/30


/home/oem/Documents/github/magnimind_projects/magpenv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1739610820.049137    6564 service.cc:148] XLA service 0x7b3fac01d700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739610820.049161    6564 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Laptop GPU, Compute Capability 8.6
2025-02-15 01:13:40.064688: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1739610820.143523    6564 cuda_dnn.cc:529] Loaded cuDNN version 90300


  6/703 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.5053 - loss: 1.0900

I0000 00:00:1739610820.669436    6564 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


703/703 ━━━━━━━━━━━━━━━━━━━━ 27s 36ms/step - accuracy: 0.6258 - loss: 0.6614 - val_accuracy: 0.7364 - val_loss: 0.5330
Epoch 2/30
  1/703 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7188 - loss: 0.5505

/home/oem/Documents/github/magnimind_projects/magpenv/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


703/703 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7188 - loss: 0.5505 - val_accuracy: 0.7332 - val_loss: 0.5400
Epoch 3/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 25s 36ms/step - accuracy: 0.7176 - loss: 0.5533 - val_accuracy: 0.7512 - val_loss: 0.5085
Epoch 4/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6875 - loss: 0.5288 - val_accuracy: 0.7512 - val_loss: 0.5093
Epoch 5/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 25s 36ms/step - accuracy: 0.7398 - loss: 0.5218 - val_accuracy: 0.7420 - val_loss: 0.5103
Epoch 6/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8438 - loss: 0.5772 - val_accuracy: 0.7440 - val_loss: 0.5082
Epoch 7/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 25s 36ms/step - accuracy: 0.7615 - loss: 0.4979 - val_accuracy: 0.7640 - val_loss: 0.5194
Epoch 8/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7812 - loss: 0.4291 - val_accuracy: 0.7680 - val_loss: 0.5076
Epoch 9/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 25s 36ms/step - accuracy: 0.7673 - loss: 0.4836 - val_accuracy: 0.768